In [7]:
%run -i '/home/.Turing/TuringCredentialsAccess.py3'
%run -i '/home/.Turing/TuringLabFunctions.py3'

------------- Turing Lab Data Science --------------------
------ Funções carregadas em memória com sucesso ---------
--------- Data da última atualização: --------------------
--------------- 17/08/2019 -------------------------------
----------------------------------------------------------
--- Consulte o manual de Biblioteca de Funções Turing: ---


----------------------------------------------------------


#### Mapear bibliotecas Python necessárias

In [8]:
import pandas as pd
import numpy as np

In [9]:
#---------- Definição dos Parâmetros do projeto ---------------------------------
nm_bucket = 'turing-bkt-treinamentos'
id_empresa = '1022'

# Id do projeto parametrizado no Turing (o mesmo valor dado ao campo Id Projeto do Timeline)
id_projeto = '10001'

# Nome da variável resposta (quando houve)
targetname = 'Survived'

# Delimitador da tabela 
abt_delimiter = ','

In [10]:
################### Não alterar este parágrafo #####################################
#---------- Gerando Credenciais para Acesso Seguro ---------------------------------
id_modelo = id_projeto
schema = 'EMP_'+ id_empresa +'_Proj_'+ id_projeto
S3fs,S3session,S3client,S3resource = TuringUsersCredentialsControl(id_empresa)
schema_rds = 'EMP_'+id_empresa+'_Proj_'+id_projeto
mysq_conn,engine = TuringRDSManagementConn(id_empresa,schema_rds,turing_adm = 'N')
path_s3_wfile = 's3://'+nm_bucket+'/Projetos/'+'Projeto_'+id_projeto+'/Modelo_Id_'+id_projeto+'/Modeltrain/'
s3_path = path_s3_wfile
####################################################################################

Acesso RDS Turing Treinamentos


#### Leitura da tabela desenvolvimento (holdout) escorada pelo Turing 

In [11]:
nm_file_desenv_holdout = 'scored_best_model_desenv.csv'
path_ = 'Projetos/'+'Projeto_'+id_projeto+'/Modelo_Id_'+id_projeto+'/Modeltrain/'
path_file = path_+str(nm_file_desenv_holdout)
desenv_df_ho = TuringReadS3CSVFile(nm_bucket,path_file,sep=abt_delimiter)

desenv_df_ho.rename( columns={'Unnamed: 0':'Id'}, inplace=True )
desenv_df_ho.shape

(694, 10)

In [12]:
desenv_df_ho.head()

,Id,Fare,Age,Pclass,FamilySize,TGD_Titulo_Mr,P_Target_0,P_Target_1,C_Target,target_t_bin
0,793,-0.030371,0.000000,-1.566107,-0.560975,1,0.591855,0.408145,0,0
1,678,0.295895,1.023561,0.827377,3.779968,0,0.952726,0.047274,0,0
2,368,-0.492378,0.000000,0.827377,-0.560975,0,0.281630,0.718370,1,1
3,569,-0.490280,0.177063,0.827377,-0.560975,1,0.903647,0.096353,0,1
4,443,-0.386671,-0.130754,-0.369365,-0.560975,0,0.113139,0.886861,1,1


#### Leitura da tabela validaçao (holdout) escorada pelo Turing 

In [13]:
nm_file_valid_holdout = 'scored_best_model_valid.csv'
path_ = 'Projetos/'+'Projeto_'+id_projeto+'/Modelo_Id_'+id_projeto+'/Modeltrain/'
path_file = path_+str(nm_file_valid_holdout)
valid_df_ho = TuringReadS3CSVFile(nm_bucket,path_file,sep=abt_delimiter)

valid_df_ho.rename( columns={'Unnamed: 0':'Id'}, inplace=True )
valid_df_ho.shape

(197, 10)

In [14]:
valid_df_ho.head()

,Id,Fare,Age,Pclass,FamilySize,TGD_Titulo_Mr,P_Target_0,P_Target_1,C_Target,target_t_bin
0,665,0.831478,0.177063,-0.369365,0.679295,1,0.457381,0.542619,1,0
1,703,-0.492545,-0.361618,0.827377,-0.560975,1,0.863568,0.136432,0,0
2,306,1.584179,0.000000,-1.566107,-0.560975,0,0.037639,0.962361,1,1
3,127,-0.504626,-0.438572,0.827377,-0.560975,1,0.960632,0.039368,0,1
4,247,-0.356469,-0.438572,-0.369365,0.679295,0,0.383123,0.616877,1,1


#### Calculando KS - Desenvolvimento

In [15]:
KS_df,KS_value_d = TuringEvaluateModels(desenv_df_ho,'P_Target_0','target_t_bin')
KS_df.head(10)

,min_scr,max_scr,bads,goods,total,odds,bad_rate,ks,max_ks
0,0.000553,0.051173,70,0,70,0.00,100.00%,26.92,
1,0.053458,0.140420,65,4,69,0.06,94.20%,51.00,
2,0.146025,0.344361,60,9,69,0.15,86.96%,72.00,
3,0.350424,0.658863,39,34,73,0.87,53.42%,79.17,<== KS ==
4,0.667823,0.814971,12,54,66,4.50,18.18%,71.34,
5,0.815290,0.873996,8,66,74,8.25,10.81%,59.21,
6,0.874009,0.922701,5,60,65,12.00,7.69%,47.31,
7,0.922934,0.957338,1,69,70,69.00,1.43%,31.80,
8,0.957358,0.977443,0,68,68,inf,0.00%,16.13,
9,0.977556,0.998669,0,70,70,inf,0.00%,0.00,


#### Calculando KS - Validação

In [16]:
KS_df,KS_value_v = TuringEvaluateModels(valid_df_ho,'P_Target_0','target_t_bin')
KS_df.head(10)

,min_scr,max_scr,bads,goods,total,odds,bad_rate,ks,max_ks
0,0.000553,0.051386,18,2,20,0.11,90.00%,20.21,
1,0.052181,0.074608,18,2,20,0.11,90.00%,40.42,
2,0.075651,0.281630,16,6,22,0.38,72.73%,54.72,
3,0.289699,0.550604,10,7,17,0.70,58.82%,60.83,<== KS ==
4,0.559137,0.741244,8,14,22,1.75,36.36%,58.41,
5,0.745804,0.792745,5,12,17,2.40,29.41%,54.07,
6,0.816737,0.887868,3,17,20,5.67,15.00%,42.95,
7,0.890443,0.935080,2,17,19,8.50,10.53%,30.60,
8,0.937508,0.971250,2,18,20,9.00,10.00%,17.39,
9,0.972426,0.997216,0,20,20,inf,0.00%,0.00,


#### Calculando Gini - Desenvolvimento

In [17]:
gini_desenv = TuringGini(desenv_df_ho,'target_t_bin','P_Target_1')
gini_desenv*100

91.75999999999999

#### Calculando Gini - Validação

In [18]:
gini_valid = TuringGini(valid_df_ho,'target_t_bin','P_Target_1')
gini_valid*100

75.96000000000001

#### Consolidando Metricas 

In [19]:
metricas = [['Desenv', KS_value_d,gini_desenv], ['Valid', KS_value_v,gini_valid]]
df = pd.DataFrame(metricas, columns = ['ABT','KS', 'Gini']) 
df

,ABT,KS,Gini
0,Desenv,79.17,0.9176
1,Valid,60.83,0.7596
